In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)
# OOM : Out Of Memory

In [2]:
# 载入数据集
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# 每批次训练数据大小 和 批次大小
batch_size = 100
n_batch = mnist.train.num_examples // batch_size


# 初始化权值
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev = 0.1) # 生成一个截断的正态分布
    return tf.Variable(initial)

# 初始化偏置
def bias_variable(shape):
    initial = tf.constant(0.1,shape = shape) # 生成一个截断的正态分布
    return tf.Variable(initial)

# 卷积层
# x 是输入一个tensor 四维 [batch批次，长，宽，in_channels通道数（彩色通道数就是3）]
# W 是滤波器filter 或者叫卷积核kernel 四维 [滤波器长，宽，输入通道数，输出通道数]
# strides步长 要求strides=[0]和strides=[3]都是1 strides=[1]代表x方向步长  strides=[2]代表y方向步长
# padding填补的方式 SAME会在外面补0  VALID就是不会补0
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

# 池化层
# ksize是窗口大小 [1,x,y,1]  strides步长同上
def max_pooling_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

# 给输入数据预留位置
x = tf.placeholder(tf.float32, shape = [None,784])
y = tf.placeholder(tf.float32, shape = [None,10])
keep_prob = tf.placeholder(tf.float32)    # 神经元的输出概率

# 改变x的格式转换为4D的向量 [batch,in_height,in_weight,in_channels]
x_image = tf.reshape(x,[-1,28,28,1])

# 初始化第一个卷积层的权重和偏置
W_conv1 = weight_variable([5,5,1,32])        # 卷积是5×5的采样窗口 1是x_image的in_channels是1维灰度图像 32是使用32个卷积核得到32个特征图
b_conv1 = bias_variable([32])                                                     # 每一个卷积核只需要一个偏置值 32个卷积核就是32个偏置值
# 把x_image和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) + b_conv1)                          # 经过卷积操作再加上偏置值 再激活 得到第一个卷积层的结果
h_pool1 = max_pooling_2x2(h_conv1)                                                                   #进行max-pooling  得到第一个池化结果


# 初始化第二个卷积层的权重和偏置
W_conv2 = weight_variable([5,5,32,64])                                    # 5×5的采样窗口 传入32个特征图 用64个卷积核得到 输出64个特征图
b_conv2 = bias_variable([64])
# 把x_image和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
h_pool2 = max_pooling_2x2(h_conv2)                                                                   #进行max-pooling
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])              # 把池化层2的输出扁平化为1维 是转置吗？ 

# 读入（reshape）28×28的图片 -->  1卷积结果（方式='SAME'补0）28×28  -->  1池化结果14×14  
#                             -->  2卷积结果（方式='SAME'补0）14×14  -->  2池化结果7×7  
# 经过上述操作 得到64张7×7的平面




# 两个全连接层     因为它是全连接层 所以它就只有两个维度

W_fc1 = weight_variable([7*7*64,1024])  # 把上面那个平面展开 要7*7*64个神经元 全连接层一共有1024个神经元
b_fc1 = bias_variable([1024])           # 1024个偏置
# h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1) + b_fc1)  # 求第一个全连接层的输出


# keep_prob = tf.placeholder(tf.float32)    # 神经元的输出概率
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

W_fc2 = weight_variable([1024,10])  # 把上面那个平面展开 要7*7*64个神经元 全连接层一共有1024个神经元
b_fc2 = bias_variable([10])           # 1024个偏置

# 预测值
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)




# 交叉熵代价函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y,logits = prediction))
# 优化器
train = tf.train.AdamOptimizer(0.0001).minimize(loss)

# 定义求准确率的方法
correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))     # 返回 Ture False 求出最大的数在的位置 看它俩返回的位置一样不
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))                                        # cast是强制类型转换 然后 求均值

# 初始化变量
init = tf.global_variables_initializer()


with tf.Session() as sess:
    sess.run(init)
    for epoch in range(21):                                                                                              # 两层循环是训练
        train_accuracy_rate = 0
        test_accuracy_rate = 0         
        for batch in range(n_batch):                               # 每次训练 是从第n个批次里取1个数据 循环结束 所有批次中的数据被训练一次           
            batch_data,batch_tag = mnist.train.next_batch(batch_size)   # 从train集中选取batch_size个训练数据 next_batch(我觉得这是bagging 错)         
#             sess.run(train,feed_dict = {x:batch_data,y:batch_tag,keep_prob:1.0})                            # 获取一个训练数据 这儿就训练一次   
#             train_accuracy_rate += sess.run(accuracy, feed_dict={x:batch_data,y:batch_tag,keep_prob:1.0})
            train_acc_rate,_ = sess.run([accuracy,train],feed_dict = {x:batch_data,y:batch_tag,keep_prob:1.0}) 
            train_accuracy_rate = train_accuracy_rate + train_acc_rate            
            batch_test_xs,batch_test_ys = mnist.test.next_batch(batch_size)
            test_accuracy_rate += sess.run(accuracy, feed_dict={x: batch_test_xs, y: batch_test_ys,keep_prob:1.0})         
        print('第' + str(epoch + 1) + '次，train准确率 ' + str(round(train_accuracy_rate/n_batch,6)) +
                                          ' test准确率 ' + str(round(test_accuracy_rate/n_batch,6)))
tf.logging.set_verbosity(old_v)



# with tf.Session() as sess:
#     sess.run(init)
#     for epoch in range(11):                                                                                              # 两层循环是训练
#         for batch in range(n_batch):                               # 每次训练 是从第n个批次里取1个数据 循环结束 所有批次中的数据被训练一次
#             batch_data,batch_tag = mnist.train.next_batch(batch_size)   # 从train集中选取batch_size个训练数据 next_batch(我觉得这是bagging 错)
#             sess.run(train,feed_dict = {x:batch_data,y:batch_tag,keep_prob:1.0})                            # 获取一个训练数据 这儿就训练一次   
#         train_accuracy_rate = 0
#         test_accuracy_rate = 0        
#         if epoch % 2 == 0:
#             for batch in range(n_batch):
#                 batch_test_xs,batch_test_ys = mnist.test.next_batch(batch_size)
#                 test_accuracy_rate += sess.run(accuracy, feed_dict={x: batch_test_xs, y: batch_test_ys,keep_prob:1.0})
#                 batch_train_xs,batch_train_ys = mnist.train.next_batch(batch_size)
#                 train_accuracy_rate += sess.run(accuracy, feed_dict={x: batch_train_xs, y: batch_train_ys,keep_prob:1.0})
#             print('第' + str(epoch + 1) + '次，train准确率 ' + str(train_accuracy_rate/(batch+1)) + ' test准确率 ' + str(test_accuracy_rate/(batch+1)))

            
# tf.logging.set_verbosity(old_v)    
    

    
#          # 每一次大循环 看一下准确率
#         if epoch % 1 == 0:
# #            train_accuracy_rate = sess.run(accuracy,feed_dict = {x:mnist.train.images,y:mnist.train.labels,keep_prob:1.0})# 训练集正确率
#             test_accuracy_rate = sess.run(accuracy,feed_dict = {x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})   # 测试集正确率
# #             print('第' + str(epoch + 1) + '次，train准确率 ' + str(train_accuracy_rate) + ' test准确率 ' + str(test_accuracy_rate))
#             print('第' + str(epoch + 1) + '次，test准确率 ' + str(test_accuracy_rate))

# tf.logging.set_verbosity(old_v)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
第1次，train准确率 0.720073 test准确率 0.725473
第2次，train准确率 0.932727 test准确率 0.937055
第3次，train准确率 0.971255 test准确率 0.9716
第4次，train准确率 0.978855 test准确率 0.9776
第5次，train准确率 0.982236 test准确率 0.980855
第6次，train准确率 0.985491 test准确率 0.981836
第7次，train准确率 0.987764 test准确率 0.984018
第8次，train准确率 0.9892 test准确率 0.985327
第9次，train准确率 0.989964 test准确率 0.986
第10次，train准确率 0.990982 test准确率 0.986745
第11次，train准确率 0.992364 test准确率 0.986673
第12次，train准确率 0.993255 test准确率 0.987873
第13次，train准确率 0.993691 test准确率 0.987709
第14次，train准确率 0.994364 test准确率 0.988091
第15次，train准确率 0.995055 test准确率 0.989145
第16次，train准确率 0.995655 test准确率 0.988927
第17次，train准确率 0.995909 test准确率 0.989309
第18次，train准确率 0.996182 test准确率 0.989909
第19次，train准确率 0.996327 test准确率 0.989927
第20次，train准确率 0.996618 test准确率 0.989455
第21次，train准确率 0.997 test

In [3]:
batch

549

In [4]:
n_batch

550